In [1]:
# Import necessary libraries
import pandas as pd               # For data manipulation using DataFrames
import numpy as np                # For numerical operations
import matplotlib.pyplot as plt   # For data visualization
import os                         # For operating system-related tasks
import joblib                     # For saving and loading models
import hopsworks                  # For getting access to hopsworks



# Import specific modules from scikit-learn
from sklearn.preprocessing import StandardScaler, OneHotEncoder   # For data preprocessing
from sklearn.metrics import accuracy_score                        # For evaluating model accuracy

In [2]:
from feature_pipeline import tesla_fg
from feature_pipeline import news_sentiment_fg

         date  1. open  2. high    3. low  4. close    5. volume ticker
0  2024-05-03   182.10   184.78  178.4200    181.19   75491539.0   TSLA
1  2024-05-02   182.86   184.60  176.0200    180.01   89148041.0   TSLA
2  2024-05-01   182.00   185.86  179.0100    179.99   92829719.0   TSLA
3  2024-04-30   186.98   190.95  182.8401    183.28  127031787.0   TSLA
4  2024-04-29   188.42   198.87  184.5400    194.05  243869678.0   TSLA
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/693399
Connected. Call `.close()` to terminate connection gracefully.
Index(['date', 'open', 'high', 'low', 'close', 'volume', 'ticker'], dtype='object')
2024-05-06 13:44:59,122 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/

Uploading Dataframe: 0.00% |          | Rows 0/3486 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: tesla_stock_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/693399/jobs/named/tesla_stock_2_offline_fg_materialization/executions
2024-05-06 13:45:08,516 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/693399/fs/689222/fg/785786


Uploading Dataframe: 0.00% |          | Rows 0/66 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: news_sentiment_updated_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/693399/jobs/named/news_sentiment_updated_2_offline_fg_materialization/executions


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [4]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/693399
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
def create_stocks_feature_view(fs, version):

    # Loading in the feature groups
    tesla_fg = fs.get_feature_group('tesla_stock', version=1)
    news_sentiment_fg = fs.get_feature_group('news_sentiment_updated', version=1)

    # Define the query
    ds_query = tesla_fg.select(['date', 'open', 'ticker'])\
        .join(news_sentiment_fg.select(['sentiment']))

    # Create the feature view
    feature_view = fs.create_feature_view(
        name='tesla_stocks_fv',
        query=ds_query,
        labels=['open']
    )

    return feature_view, tesla_fg

In [10]:
try:
    feature_view = fs.get_feature_view("tesla_stocks_fv", version=1)
    tesla_fg = fs.get_feature_group('tesla_stock', version=1)
except:
    feature_view, tesla_fg = create_stocks_feature_view(fs, 1)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/693399/fs/689222/fv/tesla_stocks_fv/version/1


In [11]:
def fix_data_from_feature_view(df,start_date,end_date):
    df = df.sort_values("date")
    df = df.reset_index()
    df = df.drop(columns=["index"])

    # Create a boolean mask for rows that fall within the date range
    mask = (pd.to_datetime(df['date']) >= pd.to_datetime(start_date)) & (pd.to_datetime(df['date']) <= pd.to_datetime(end_date))
    len_df = np.shape(df)
    df = df[mask] # Use the boolean mask to filter the DataFrame
    print('From shape {} to {} after cropping to given date range: {} to {}'.format(len_df,np.shape(df),start_date,end_date))

    # Get rid off all non-business days
    isBusinessDay, is_open = extract_business_day(start_date,end_date)
    is_open = [not i for i in is_open] # Invert the mask to be able to drop all non-buisiness days

    filtered_df = df.drop(df[is_open].index) # Use the mask to filter the rows of the DataFrame
    print('From shape {} to {} after removing non-business days'.format(np.shape(df),np.shape(filtered_df)))
    print(filtered_df)
    
    return filtered_df

In [7]:
#def create_stocks_feature_view(fs, version):

    #Loading in the feature groups
#    tesla_fg = fs.get_feature_group('tesla_stock', version = 3)
#    news_sentiment_fg = fs.get_feature_group('news_sentiment_updated', version = 2)

#    ds_query = tesla_fg.select(['date','open', 'ticker'])\
#        .join(news_sentiment_fg.select_except(['ticker','time', 'amp_url', 'image_url']))
    
#    return (fs.create_tesla_feature_view(
#        name = 'tsla_stocks_fv',
#        query = ds_query,
#        labels=['ticker']
#    ), tesla_fg)

In [8]:
#try:
#    feature_view = fs.get_feature_view("tsla_stocks_fv", version=1)
#    tesla_fg = fs.get_feature_group('tesla_stock', version=3)
#except:
#    feature_view, tesla_fg = create_stocks_feature_view(fs, 1)